# Schedule Linear Program

In [ ]:
#call the method to make the .dat file from excel
from DataFileFormatting import read_sheet
read_sheet()
import time
from pyomo.environ import *
from pyomo.core import Var
#Declare the model
model=AbstractModel()
print("hello")


hello


## Sets, Parameters, and Variables

In [2]:

#sets
model.Worker = Set() 	#Number of available workers to work
model.Days = Set()		#Days of the week Monday through Sunday
model.Time= Set()		#Hours that the Warrior Factory is open in military time
model.Events= Set()		#Types of events that occur through out the day
model.YvalueTime=Set()	#One hour before actually starting/working (Time-1) (For math purposes)



#Parameter
model.Prestige= Param(model.Worker)								#1 if a worker is of high prestige, 0 if a worker is of average prestige
model.Type= Param(model.Worker)									#1 if a worker is full-time, 0 if a worker is part-time
model.StaffingReq=Param(model.Events) 							#Number of employees required to work during each event
model.StaffingPrestige=Param(model.Events)						#Required number of employees with high prestige status to work at each event
model.Availability= Param(model.Worker,model.Days, model.Time)	#1 if the employee is available to work on a given day at a given hour, 0 if they are unavailable
model.c=Param(model.Worker)										#Cost per hour for employees
model.ScheduleReq=Param(model.Days, model.Time) 				#Schedule of events occurring each day at each hour Warrior Factory is open 

#Decision variables
model.x = Var(model.Worker,model.Days, model.Time,within=Binary) 		#1 if a worker will work during a given shift, 0 if they will not be working
model.y = Var(model.Worker,model.Days, model.YvalueTime,within=Binary)	#1 if the worker is allowed to work start working on the next hour (used for math purposes)
model.r=Var(model.Worker,within=NonNegativeReals) 						#Amount of regular time hours an employee is working in a week
model.o=Var(model.Worker,within=NonNegativeReals)						#Amount of OT hours an employee is working in a week


In [3]:

#Objective Function minimizing cost in wages of the workers for the given week, employees
#receive a regular time salary, and time and a half for overtime
def objective_rule(model):
    return sum(model.c[i]*model.r[i]+1.5*model.c[i]*model.o[i]  for i in model.Worker)
model.minCost = Objective(rule=objective_rule,sense=minimize)


## Relate hours worked to Overtime and Regular time

In [4]:

#constraints

#A worker can only work 45 hours of regular time
def MaxRegularTime(model,i):
    return model.r[i]<=45
model.MaxRegularTimeConstraint=Constraint(model.Worker, rule=MaxRegularTime)

#The sum of a workers regular time and overtime equals the total number of hours worked
def OvertimeRelation(model,i):
    return (model.o[i]+model.r[i]==sum(model.x[i,j,k] for j in model.Days for k in model.Time))
model.OvertimeRelationConstraint=Constraint(model.Worker, rule=OvertimeRelation)



## Worker Rules

In [5]:
#The maximum number of hours an employee is allowed to work per week
def MaxTimePerWorkerRule(model,i):
    if model.Type[i] == 0:			#A part-time worker can work up to 29.5 hours per week
        return (sum(model.x[i,j,k] for j in model.Days for k in model.Time) <= 29.5)
    if model.Type[i]==1:			#A full-time worker can work up to 60 hours per week
        return (sum(model.x[i,j,k] for j in model.Days for k in model.Time) <= 60)
model.MaxTimePerWorker= Constraint(model.Worker,rule=MaxTimePerWorkerRule)

#The minimum number of hours an employee is allowed to work per week
def MinTimeRule(model,i):
    if model.Type[i] == 1:			#A full-time worker must work at least 40 hours per week
        return (sum(model.x[i,j,k] for j in model.Days for k in model.Time)>= 40)
    if model.Type[i] == 0:			#A part-time does not have to work in the given week
        return (sum(model.x[i,j,k] for j in model.Days for k in model.Time) >= 0)
model.MinTimePerWorker= Constraint(model.Worker,rule=MinTimeRule)

# If a worker is available, they may work; if they are not available they will not be scheduled to work
def AvailabilityRule(model,i,j,k):
    if model.Availability[i,j,k] == 0:
        return (model.x[i,j,k] == 0)
    if model.Availability[i,j,k] == 1:
        return (model.x[i,j,k] <= 1)
model.AvailabilityRule = Constraint(model.Worker,model.Days,model.Time, rule=AvailabilityRule)


## Schedule Requirements

In [6]:

#parties require at least one high prestige worker, all other events do not need to have a high prestige worker
def PrestigeRequirements(model,j,k):
    if model.ScheduleReq[j,k] == 'Party':
        return (sum(model.Prestige[i]*model.x[i,j,k] for i in model.Worker) >= 1)
    if model.ScheduleReq[j,k] == 'Class':
        return (sum(model.Prestige[i]*model.x[i,j,k] for i in model.Worker) >= 0)
    if model.ScheduleReq[j,k] == 'AdultOpenGym':
        return (sum(model.Prestige[i]*model.x[i,j,k] for i in model.Worker) >= 0)
    if model.ScheduleReq[j,k] == 'KidOpenGym':
        return (sum(model.Prestige[i]*model.x[i,j,k] for i in model.Worker) >= 0)
    if model.ScheduleReq[j,k] == 'StartDay':
        return (sum(model.Prestige[i]*model.x[i,j,k] for i in model.Worker) >= 0)
model.Prestige_Requirements= Constraint(model.Days,model.Time, rule=PrestigeRequirements)

#Number of workers required to work each event type
def wRequirements(model,j,k):
    if model.ScheduleReq[j,k] == 'Party':
        return (sum(model.x[i,j,k] for i in model.Worker) >= 6)
    if model.ScheduleReq[j,k] == 'Class':
        return (sum(model.x[i,j,k] for i in model.Worker) >= 3)
    if model.ScheduleReq[j,k] == 'AdultOpenGym':
        return (sum(model.x[i,j,k] for i in model.Worker) >= 4)
    if model.ScheduleReq[j,k] == 'KidOpenGym':
        return (sum(model.x[i,j,k] for i in model.Worker) >= 6)
    if model.ScheduleReq[j,k] == 'StartDay':				#used to skip constraint- non-binding
        return (sum(model.x[i,j,k] for i in model.Worker) >= 1)
model.w_Requirements= Constraint(model.Days,model.Time, rule=wRequirements)


## Shift rules

In [7]:
#Each employee can only start work once per day
def ConsecutiveYRule(model,i,j):
    return(sum(model.y[i,j,k] for k in model.YvalueTime)<=1)
model.ConsecutiveYConstraint = Constraint(model.Worker,model.Days, rule=ConsecutiveYRule)

#An employee must work a full shift, full-time employees work 8 hr shifts and part time employees work 4 hour shifts
def ConsecutiveHourRule (model,i,j,k):
    if model.Type[i] == 0:																		#the for loop for set kk counts the 4 previous hours and verifies that the worker is working these hours, 
        return model.x[i,j,k] - sum(model.y[i,j,kk] for kk in (range(max(7,k-4),k)))==0			#and thus equals 0 because a part-time employee can only start work once				
    if model.Type[i] == 1:																				#the for loop for set kk counts the 8 previous hours and verifies that the worker is working these hours,	
        return model.x[i,j,k] - sum(model.y[i,j,kk] for kk in (range(max(7,k-8),k)))==0					#and thus equals 0 because a full-time employee can only start work once
model.ConsecutiveHourConstraint= Constraint(model.Worker, model.Days, model.Time, rule=ConsecutiveHourRule)

#The latest time each type of employee can begin work; an employee can't start working if there are not enough hours left in the day to complete a full shift
def LateDayRule (model,i,j):
    if model.Type[i] == 0:
        return model.y[i,j,18]+model.y[i,j,18+1]+model.y[i,j,18+2]+model.y[i,j,18+3]==0
    if model.Type[i] == 1:
        return model.y[i,j,14]+model.y[i,j,14+1]+model.y[i,j,14+2]+model.y[i,j,14+3]+model.y[i,j,14+4]+model.y[i,j,14+5]+model.y[i,j,14+6]+model.y[i,j,14+7]==0
model.LateDayConstraint= Constraint(model.Worker, model.Days, rule=LateDayRule)



#At least one person must open Warrior Factory at 8AM
def WorkersMustOpenRule(model, j):
    return (sum(model.y[i,j,7] for i in model.Worker)>=1)
model.WorkerMustOpen= Constraint(model.Days,rule=WorkersMustOpenRule)



## Running Model and Formatting

In [8]:

#Running the model in PYOMO
data = DataPortal()
data.load(filename="excelData.dat",model=model)
optimizer=SolverFactory('glpk')
instance=model.create_instance(data)
optimizer.solve(instance)
instance.display(filename="pyomoFormatOutput.txt")


In [9]:

#creating a more readable file 
file1 = open("ScheduleOutput.txt","w+")

print((time.strftime("%H:%M.%S")))
print("Weekly Cost= $" + str(instance.minCost.expr()) + "0" )
file1.write(" /n Weekly Cost= ")
file1.write("$" + str(instance.minCost.expr()) + "0" +'\n\n')	#the objective function value

for v in instance.component_objects(Var, active =True):         #gets the instance of the variables, and prints the x values with a value of 1
    varobject = getattr(instance,str(v))
    if str(v) =="y":		
        file1.write("Schedule:" +'\n')
        for index in varobject:
            if str(varobject[index].value) == ("1.0"):          
                file1.write(str(index[0]+" will work "+index[1]+" at "+str(index[2]+1)))
                print(str(index[0]+" - "+index[1]+" - "+str(index[2]+1)))
                file1.write('\n')      

file1.close() 


23:05.55
Weekly Cost= $9140.00
Liz - M - 9
Liz - F - 9
Liz - R - 9
Liz - S - 12
Liz - Su - 8
Tim - M - 14
Tim - R - 14
Tim - T - 14
Tim - S - 14
Tim - W - 14
Zoe - R - 14
Zoe - T - 14
Zoe - S - 14
Zoe - W - 14
Zoe - Su - 14
Sam - M - 14
Sam - F - 14
Sam - R - 14
Sam - T - 14
Sam - W - 9
Pat - M - 14
Pat - F - 14
Pat - R - 14
Pat - T - 9
Pat - W - 9
Dan - Su - 16
Kim - F - 9
Kim - T - 8
Kim - S - 8
Kim - W - 8
Kim - Su - 8
Leo - M - 9
Leo - F - 9
Leo - R - 9
Leo - S - 9
Leo - W - 14
Asa - M - 16
Asa - F - 16
Asa - R - 16
Asa - T - 16
Asa - S - 18
Asa - W - 16
Asa - Su - 18
Bob - S - 8
Bob - Su - 8
Ken - M - 8
Ken - F - 8
Ken - R - 8
Ken - S - 10
Ken - Su - 10
Ben - M - 16
Ben - F - 16
Ben - R - 16
Ben - T - 16
Ben - S - 18
Ben - W - 16
Ben - Su - 18
Cam - S - 16
Cam - Su - 10
Max - M - 9
Max - R - 9
Max - T - 9
Max - S - 11
Max - Su - 12
Meg - F - 14
Meg - T - 9
Meg - S - 11
Meg - W - 9
Meg - Su - 11
Tom - M - 14
Tom - F - 14
Tom - T - 14
Tom - W - 14
Tom - Su - 14
